In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torch.autograd as autograd


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
print(device)


cuda


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [0]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [0]:
def filterLong(pairs):
    pairList = []
    MAX_LENGTH = 10
    for i in range(len(pairs)):
        if(len(pairs[i])>1):
          sent0 = pairs[i][0]
          sent1 = pairs[i][1]
          
        if(" " in sent0):
            temoSent0 = sent0.split(" ")
        if(" " in sent1):
            temoSent1 = sent1.split(" ")

        if((len(temoSent0) <= MAX_LENGTH) and (len(temoSent1) <= MAX_LENGTH)):
          pairList.append(pairs[i])
    return pairList

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('/content/drive/My Drive/translate/question_answer.txt').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [0]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [7]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterLong(pairs)
    twoConv = []
    for i in range(len(pairs)):
        if(len(pairs[i]) >1):
            twoConv.append(pairs[i])


    pairs = filterPairs(twoConv)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        if(len(pair) > 1):
          input_lang.addSentence(pair[0])
          output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'gar', True)
print(random.choice(pairs))

Reading lines...
Read 699281 sentence pairs
Trimmed to 12879 sentence pairs
Counting words...
Counted words:
gar 80
eng 81
['thanks .', 'you re welcome .']


In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        #print("coming here")
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        #embedded = self.dropout(embedded)
        #print("Embd size-->",embedded.size)
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [0]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [0]:
## With attention

teacher_forcing_ratio = 0.5
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # if use_teacher_forcing:
    #     # Teacher forcing: Feed the target as the next input
    #     for di in range(target_length):
    #         decoder_output, decoder_hidden, decoder_attention = decoder(
    #             decoder_input, decoder_hidden, encoder_outputs)
    #         loss += criterion(decoder_output, target_tensor[di])
    #         decoder_input = target_tensor[di]  # Teacher forcing

    # else:
        # Without teacher forcing: use its own predictions as the next input
    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [0]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=10, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)
    #return plot_losses

In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]



        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1], encoder_outputs 

In [0]:
def evaluateRandomlyTranslate(encoder, decoder, n=100):
    encoder_outputs = []
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions, en_out = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

        #print(en_out)
        # maxVal = max(en_out)
        # minVal = min(en_out)

        # print("Max value->",maxVal)
        # print("Min value->",minVal)   

        encoder_outputs.append(en_out)

    return encoder_outputs

In [84]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
encoder1.load_state_dict(torch.load('/content/drive/My Drive/translate/encoder_tran_en_de_new_qus_ans.dict'))
attn_decoder1.load_state_dict(torch.load('/content/drive/My Drive/translate/decoder_with_attention_tran_en_de_new_qus_ans.dict'))

encoder_outputs = evaluateRandomlyTranslate(encoder1, attn_decoder1)




> eight .
= they re all so cute .
< they re all so cute . <EOS>

> what are they doing ?
= they are admiring the beauty of the birds .
< they are admiring the beauty of the birds . <EOS>

> what are they doing ?
= they are admiring the beauty of the birds .
< they are admiring the beauty of the birds . <EOS>

> how has mass media changed recently ?
= they re more modern and user friendly .
< they re more modern and user friendly . <EOS>

> thanks .
= you re welcome .
< you re welcome . <EOS>

> no i always back up my files .
= you re smart .
< you re smart . <EOS>

> what are they doing ?
= they are admiring the beauty of the birds .
< they are admiring the beauty of the birds . <EOS>

> it s supposed to be unlucky .
= you re supposed to stay home all day .
< you re supposed to stay home all day . <EOS>

> my husband died .
= i m sorry for you .
< i m sorry for you . <EOS>

> i mean you re wasting your life .
= i m having fun .
< i m having fun . <EOS>

> where is your brother studying

In [22]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
encoder1.load_state_dict(torch.load('/content/drive/My Drive/translate/encoder_tran_en_de_new_qus_ans.dict'))
attn_decoder1.load_state_dict(torch.load('/content/drive/My Drive/translate/decoder_with_attention_tran_en_de_new_qus_ans.dict'))

<All keys matched successfully>

In [0]:
class Block(nn.Module):

    def __init__(self, block_dim):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(block_dim, block_dim),
            nn.ReLU(True),
            nn.Linear(block_dim, block_dim),
        )

    def forward(self, x):
        return self.net(x) + x


class Generator(nn.Module):

    def __init__(self, n_layers, block_dim):
        super().__init__()

        self.net = nn.Sequential(
            *[Block(block_dim) for _ in range(n_layers)]
        )

    def forward(self, x):
        return self.net(x)


class Critic(nn.Module):

    def __init__(self, n_layers, block_dim):
        super().__init__()

        self.net = nn.Sequential(
            *[Block(block_dim) for _ in range(n_layers)]
        )

    def forward(self, x):
        return self.net(x)

In [0]:
n_layers = 20
block_dim = 256
gp_lambda = 10
latent_dim = 256
interval = 1000
batch_size = 1
n_critic = 100
generator = Generator(n_layers, block_dim)
critic = Critic(n_layers, block_dim)
critic.to(device)
g_optimizer = optim.Adam(generator.parameters(), lr=1e-4)
c_optimizer = optim.Adam(critic.parameters(), lr=1e-4)

In [0]:
def testGan(encoder, decoder, sentence, max_length=10):
    with torch.no_grad():
        #print("1")
        input_length = sentence.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
        encoder_outputs[0: input_length,:] = sentence

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)
        
        #print("2")
        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            #print("3")
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [0]:
from numpy.random import randint

def evaluateGAN(encoder, decoder, encoder_outputs, n=100):
    latent_dim = 256
    n_layers = 20
    block_dim = 256
    output_sentence_list = []
    while(len(output_sentence_list) < 100):
        generator = Generator(n_layers, block_dim)
        generator.eval()
        generator.load_state_dict(torch.load('/content/drive/My Drive/translate/generator_question_answer.th', map_location='cpu'))
        input_tensor = [[85],[86],[109],[1362],[2],[6],[1]]
        input_tensor = torch.from_numpy(np.asarray(input_tensor)).float()
        noise = torch.from_numpy(np.random.normal(loc = 0, scale = 1, size = (10,256)))
        
        #noise = torch.from_numpy(input_tensor).float()
        #z = generator(noise)
        #print("Z shape=",z.shape)
        #print("z=", z)
        pair = random.choice(pairs)
        #print('>', pair[0])
        #print('=', pair[1])
        #print(noise)
        #output_words, attentions = testGan(encoder, decoder, encoder_outputs[i])
        output_words, attentions = testGan(encoder, decoder, noise)
        #output_words, attentions = testGan(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)

        if(output_sentence not in output_sentence_list):
          output_sentence_list.append(output_sentence)
          print('<', output_sentence)
          print('')

In [0]:
evaluateGAN(encoder1,attn_decoder1, encoder_outputs)

< i m not sure . <EOS>

< you re supposed to stay home all day . <EOS>

< you re smart . <EOS>

< you re welcome . <EOS>

< i m not ever going to leave . <EOS>

< i m a waist and user friendly . <EOS>

< i m not ever going to take bath . <EOS>

< i m fine thanks ! <EOS>

< i m having fun . <EOS>

< i m not sure and user friendly . <EOS>

< i m not sure and hospitable . <EOS>

< you re more modern and user friendly . <EOS>

< i m not ever going to <EOS>

< i m sorry sure . <EOS>

< i m a teacher . <EOS>

< i m not ever going . <EOS>

< i m not sure and user . <EOS>

< i m not sure to take bath . <EOS>

< i m not sure sure . <EOS>

< i m not sure to stay home all day .

< i m sorry for you . <EOS>

< i am following my friends to take bath . <EOS>

< i m a teacher and user friendly . <EOS>

< they re all so cute . <EOS>

< they re more modern and user friendly . <EOS>

< i m not sure to take . <EOS>

< i m fine ok well thank you . <EOS>

< they re friendly and hospitable . <EOS>

< you re